Post 방식

In [10]:
import urllib.request as req
import pandas as pd
import requests
from selenium import webdriver
import time
from datetime import datetime
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup

In [2]:
# 금 시세 테이블이 있는 url
url_target = 'https://www.koreagoldx.co.kr/main/html.php?agencyCode=&htmid=goods/gold_list.html'

[header 및 payload]

In [3]:
header = {'User-Agent':'Mozilla/5.0'}

In [4]:
payload = {"srchDt": "ALL", "type": "Au", "dataDateStart": "2008.03.11", "dataDateEnd": "2021.05.06"}

[헤더와 페이로드 포함한 페이지 요청 및 respoonse]

In [5]:
rr = requests.post(url_target, headers=header, data=payload)

[응답 페이지 소스 파싱하기]

In [11]:
soup_test = BeautifulSoup(rr.content, 'html.parser')

In [12]:
print(soup_test.prettify())

<!DOCTYPE html>
<html>
 <head>
  <title>
   한국금거래소 - 대한민국 금거래의 중심
  </title>
  <meta content="IE=edge" http-equiv="X-UA-Compatible"/>
  <meta content="text/html; charset=utf-8" http-equiv="Content-Type">
   <meta content="한국금거래소" name="author"/>
   <meta content="오늘의 금시세(금값시세), 은시세, 금매입, 관련 시세 정보 제공으로 투명한 거래에 앞장서고 있습니다.." name="description"/>
   <meta content="금시세,금값,금값시세, 금매입, 국제금값,은값,은시세,골드바,실버바, 금, 한국금, 포나인, 고가매입, 조폐공사, 금괴, 투자, 오늘의금시세, 금테크, 재테크, 선물, 금화, 은화, 은테크," name="keywords"/>
   <meta content="MTYyMDQ0MDM0NjM5ODczNzg0MDAzNDEzNTY0NjY0MjkyNDM3MTI0NTIy" name="csrf-token"/>
   <meta content="website" property="og:type"/>
   <meta content="한국금거래소" property="og:title"/>
   <meta content="https://www.koreagoldx.co.kr/data/common/snsRepresentImage.jpg" property="og:image"/>
   <meta content="https://www.koreagoldx.co.kr/main/html.php?agencyCode=&amp;htmid=goods/gold_list.html" property="og:url"/>
   <meta content="대한민국 금거래의 중심" property="og:description"/>
   <meta content="ko_KR" prope

헤더와 payload를 포함하여 페이지를 요청하였으나 우리가 필요한 테이블 데이터는  
html에 포함되어 있지 않으며 자바스크립트로 동적 구성되어 있다.

`<!-- 시세조회 스크립트 -->` 윗 부분 `<div id="example-table">` 확인

**<u>이런 경우 selenium을 이용하여 동적 크롤링하면 우리가 보는 웹 페이지의 소스 정보  
모두를 얻어올 수 있다.</u>**

selenium 

In [19]:
from selenium import webdriver

In [20]:
# 크롬 드라이버 구동 및 객체 생성

In [21]:
driver = webdriver.Chrome('C:/python_lee/chrome/chromedriver.exe')

In [22]:
url = 'https://www.koreagoldx.co.kr/main/html.php?agencyCode=&htmid=goods/gold_list.html'

In [23]:
driver.get(url)

구동 중인 크롬브라우저에서 해당 웹페이지의 필요 정보 클릭  
--> **css selector 활용하여 `전체` 클릭*

In [24]:
#contents > div.sub_content > div > div.service_cont > div.clear > ul.graph_dt_buttons > li:nth-child(6) > label

In [25]:
# 해당 원하는 전체 값을 보여주는 위치값을 복사함.

In [26]:
all_element = driver.find_element_by_css_selector('#contents > div.sub_content > div > div.service_cont > div.clear > ul.graph_dt_buttons > li:nth-child(6) > label')

In [27]:
all_element.click()

[필요 데이터 접근 가능여부 확인]

[응답 받은 페이지 소스 정보에서 find_element_by_class_name() 이용하여 필요 데이터 수집 가능한지 확인

In [28]:
data_test = driver.find_element_by_class_name('tabulator-table')

In [29]:
data_test.text

'2021.05.08284,000263,500189,800146,300\n2021.05.07284,000263,000189,400146,000\n2021.05.06282,000261,500188,300145,200\n2021.05.05279,500260,500187,600144,600\n2021.05.04280,500260,500187,600144,600\n2021.05.03278,500260,000187,200144,300\n2021.05.01276,000259,500186,900144,100\n2021.04.30273,500258,000185,800143,200\n2021.04.29276,000259,500186,900144,100\n2021.04.28275,000258,500186,200143,500\n2021.04.27275,500259,500186,900144,100\n2021.04.26276,500260,000187,200144,300\n2021.04.24277,000260,000187,200144,300\n2021.04.23278,500260,500187,600144,600\n2021.04.22279,000260,500187,600144,600'

위에는 한 페이지만. 이제부터는 다음 페이지로 넘기는 방법

In [ ]:
#example-table > div.tabulator-footer > span > button:nth-child(4)

In [31]:
click_element = driver.find_element_by_css_selector('#example-table > div.tabulator-footer > span > button:nth-child(4)')

In [32]:
click_element.click()

### [한국금거래소의 금 시세 데이터를 수집하고 시각화]

In [77]:
import pandas as pd
df = pd.DataFrame(columns = ['date'])

In [88]:
text = ""

In [89]:
i = 1
while i < 268:

    all_element = driver.find_element_by_css_selector('#contents > div.sub_content > div > div.service_cont > div.clear > ul.graph_dt_buttons > li:nth-child(6) > label')
    all_element.click()
    data_test = driver.find_element_by_class_name('tabulator-table')
    text += data_test.text + '\n' 
    click_element = driver.find_element_by_css_selector('#example-table > div.tabulator-footer > span > button:nth-child(4)')
    click_element.click()

    i = i + 1

In [91]:
#text

In [93]:
raw_data = text.split("\n")[:4002]

In [94]:
len(raw_data)

4002

In [95]:
raw_data[0][:10]

'2021.05.08'

In [96]:
raw_dict = {}
raw_dict.setdefault('date',[])
raw_dict.setdefault('buy',[])
raw_dict.setdefault('sell_24k',[])
raw_dict.setdefault('sell_18k',[])
raw_dict.setdefault('sell_14k',[])

[]

In [98]:
import urllib.request as req
import pandas as pd
import requests
from selenium import webdriver
import time
from datetime import datetime
import matplotlib.pyplot as plt

In [99]:
for data in raw_data:
    raw_dict['date'].append(data[:10])
    raw_dict['buy'].append(re.findall('\d+, \d{3}', data[10:])[0])
    raw_dict['sell_24k'].append(re.findall('\d+,\d{3}', data[10:])[1])
    raw_dict['sell_18k'].append(re.findall('\d+,\d{3}', data[10:])[2])
    raw_dict['sell_14k'].append(re.findall('\d+,\d{3}', data[10:])[3])

NameError: name 're' is not defined

In [ ]:
raw_dict

In [ ]:
raw_dict.keys()

In [ ]:
dict_keys(['date', 'buy', 'sell_24k', 'sell_18k', 'sell_14k'])

In [ ]:
import pandas as pd

In [ ]:
raw_df = pd.DataFrame(raw_dict)

In [ ]:
raw_df

In [ ]:
# 데이터 저장
raw_df.to_csv('data/examples/gold.csv', index=False)

In [ ]:
# 데이터 불러오기
raw_df = pd.read_csv('data/examples/gold.csv')

In [ ]:
gold24k = raw_df[['date', 'buy', 'sell_24k']]

In [ ]:
temp_re = lambda x:x.replace(",","")

In [ ]:
crisis_data = [
    (datetime(2008, 3, 13), 'Bear Stearns Fails'),
    (datetime(2008, 9, 15), 'Lehman Bankruptcy'),
    (datetime(2012, 12, 19), '18th President Election'),
    (datetime(2017, 5, 9), '19th President Election'),
    (datetime(2020, 2, 12), 'COVID-19 by WHO')
]